In [8]:
import tensorflow as tf
import keras
import numpy as np

from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.models import load_model
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io

tf.__version__
tf.compat.v1.disable_eager_execution()

In [9]:
# If GPU is not available: 
# GPU_USE = '/cpu:0'
# config = tf.compat.v1.ConfigProto(device_count = {"GPU": 0})


# If GPU is available: 
config = tf.compat.v1.ConfigProto()
config.log_device_placement = True
# Allow dynamic allocation of GPU memory
config.allow_soft_placement = True
# The memory is managed in blocks, and space allocation and release are performed by blocks
config.gpu_options.allocator_type = 'BFC'

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.1

# set session config
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

Device mapping: no known devices.


## Define input and output

In [10]:
input_file = "myModel.h5"

output_file = "myModel.pb"

output_node_prefix = "output_node"

In [11]:
K.set_learning_phase(0)
net_model = load_model(input_file)

num_output = 1
pred = [None]*num_output
pred_node_names = [None]*num_output
for i in range(num_output):
    print(i)
    pred_node_names[i] = output_node_prefix+str(i)
    # Return a Tensor with the same shape and contents as input
    pred[i] = tf.identity(net_model.outputs[i], name=pred_node_names[i])
print('output nodes names are: ', pred_node_names)
output_node_prefix = pred_node_names[0]
print(pred_node_names)
print(output_node_prefix)

0
output nodes names are:  ['output_node0']
['output_node0']
output_node0


In [ ]:
sess = tf.compat.v1.keras.backend.get_session()
sess.graph.as_graph_def()

## Export model

We replace all the variables in the graph with constants of the same values. This makes it possible to describe the network fully with a single GraphDef file, and allows the removal of a lot of ops related to loading and saving the variables. It is very convenient to only have a single file for deployment in production.

In [7]:
constant_graph = tf.compat.v1.graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)
graph_io.write_graph(constant_graph, "",output_file, as_text=False)

print('Saved the freezed graph at: ', (output_file))

NameError: name 'sess' is not defined

## Show input and output node

In [ ]:
g = tf.compat.v1.GraphDef()
g.ParseFromString(open(output_file, "rb").read())
s = ""
for n in g.node:
    s =s + str(n.name) + "\n"

print(s)